# Logistic Regression Consulting Project

## Binary Customer Churn

A marketing agency has many customers that use their service to produce ads for the client/customer websites. They've noticed that they have quite a bit of churn in clients. They basically randomly assign account managers right now, but want you to create a machine learning model that will help predict which customers will churn (stop buying their service) so that they can correctly assign the customers most at risk to churn an account manager. Luckily they have some historical data, can you help them out? Create a classification algorithm that will help classify whether or not a customer churned. Then the company can test this against incoming data for future customers to predict which customers will churn and assign them an account manager.

The data is saved as customer_churn.csv. Here are the fields and their definitions:

    Name : Name of the latest contact at Company
    Age: Customer Age
    Total_Purchase: Total Ads Purchased
    Account_Manager: Binary 0=No manager, 1= Account manager assigned
    Years: Totaly Years as a customer
    Num_sites: Number of websites that use the service.
    Onboard_date: Date that the name of the latest contact was onboarded
    Location: Client HQ Address
    Company: Name of Client Company
    
Once you've created the model and evaluated it, test out the model on some new data (you can think of this almost like a hold-out set) that your client has provided, saved under new_customers.csv. The client wants to know which customers are most likely to churn given this data (they don't have the label yet).

In [1]:
import findspark 
findspark.init('/home/gkouskosv/spark-2.4.5-bin-hadoop2.6/')

In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName('ConProj').getOrCreate()

In [3]:
data = spark.read.csv('customer_churn.csv',header=True,inferSchema=True)

In [4]:
data.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: timestamp (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)
 |-- Churn: integer (nullable = true)



In [5]:
data.select('Company').distinct().count()

873

In [6]:
by_comp = data.groupBy('Company').count()
by_comp.orderBy(by_comp['count'].desc()).show()

+--------------+-----+
|       Company|count|
+--------------+-----+
|Anderson Group|    4|
|    Wilson PLC|    3|
|  Williams PLC|    3|
|Perry and Sons|    2|
|     Gates Ltd|    2|
|    Walker Ltd|    2|
|   Davis Group|    2|
|      Webb PLC|    2|
|   Smith Group|    2|
|Smith and Sons|    2|
|     Evans LLC|    2|
|     Smith Inc|    2|
|  Williams LLC|    2|
|      Rice PLC|    2|
|      Soto PLC|    2|
|    Nelson LLC|    2|
|      King LLC|    2|
|     Ortiz Ltd|    2|
|     Jones LLC|    2|
|Davis and Sons|    2|
+--------------+-----+
only showing top 20 rows



As for the column 'Company', it is better to drop it than hold it. The amount of companies are 873, and each one are shown from 1 to 4 times.

In [7]:
data.select('Num_Sites').distinct().show()

+---------+
|Num_Sites|
+---------+
|      8.0|
|      7.0|
|      4.0|
|     11.0|
|     14.0|
|      3.0|
|     10.0|
|     13.0|
|      6.0|
|      5.0|
|      9.0|
|     12.0|
+---------+



Let's find out if Location column is helpful enough to be in out logistic regression model. It is better to hold only the zipcode from the whole address.

In [8]:
data.select('Location').head(5)

[Row(Location='10265 Elizabeth Mission Barkerburgh, AK 89518'),
 Row(Location='6157 Frank Gardens Suite 019 Carloshaven, RI 17756'),
 Row(Location='1331 Keith Court Alyssahaven, DE 90114'),
 Row(Location='13120 Daniel Mount Angelabury, WY 30645-4695'),
 Row(Location='765 Tricia Row Karenshire, MH 71730')]

In [9]:
from pyspark.sql.functions import split, monotonically_increasing_id

In [10]:
zip_code = [word[0][-1] for word in data.select(split('Location', ' ')).collect()]
zip_code[:5]

['89518', '17756', '90114', '30645-4695', '71730']

In [11]:
def del_minus(lst):
    without_minus = []
    for code in lst:
        if '-' in code:
            without_minus.append(code.split('-')[0])
        else:
            without_minus.append(code)
    return without_minus

In [12]:
no_minus_zip = del_minus(zip_code)

In [13]:
no_minus_zip[:5]

['89518', '17756', '90114', '30645', '71730']

In [14]:
zip_code[:5]

['89518', '17756', '90114', '30645-4695', '71730']

In [15]:
data.select('Location', split('Location', ' ')[0]).show()

+--------------------+---------------------+
|            Location|split(Location,  )[0]|
+--------------------+---------------------+
|10265 Elizabeth M...|                10265|
|6157 Frank Garden...|                 6157|
|1331 Keith Court ...|                 1331|
|13120 Daniel Moun...|                13120|
|765 Tricia Row Ka...|                  765|
|6187 Olson Mounta...|                 6187|
|4846 Savannah Roa...|                 4846|
|25271 Roy Express...|                25271|
|3725 Caroline Str...|                 3725|
|363 Sandra Lodge ...|                  363|
|Unit 8120 Box 916...|                 Unit|
|Unit 1895 Box 094...|                 Unit|
|897 Kelley Overpa...|                  897|
|11488 Weaver Cape...|                11488|
|1774 Peter Row Ap...|                 1774|
|45408 David Path ...|                45408|
|28216 Wright Moun...|                28216|
|Unit 4948 Box 481...|                 Unit|
|69203 Crosby Divi...|                69203|
|9569 Cald

In [16]:
no_minus_zip = spark.createDataFrame([(value,) for value in no_minus_zip], ['no_minus_zip'])

In [17]:
type(no_minus_zip['no_minus_zip'])

pyspark.sql.column.Column

In [18]:
zip_code[0]

'89518'

In [19]:
zip_code[:10]

['89518',
 '17756',
 '90114',
 '30645-4695',
 '71730',
 '74359',
 '87713-3460',
 '59852-6150',
 '82059',
 '51655-7561']

In [20]:
zip_code = spark.createDataFrame([(value,) for value in zip_code], ['zip_code'])

In [21]:
zip_code = zip_code.withColumn('id', monotonically_increasing_id())
data = data.withColumn('id', monotonically_increasing_id())

In [22]:
data = data.join(zip_code, on='id', how='inner').drop('id')

In [23]:
data.select('zip_code').show()

+----------+
|  zip_code|
+----------+
|     89518|
|     17756|
|     90114|
|30645-4695|
|     71730|
|     74359|
|87713-3460|
|59852-6150|
|     82059|
|51655-7561|
|     43432|
|     40249|
|     44793|
|63417-8544|
|     18782|
|54903-6698|
|40999-2369|
|     42669|
|     87064|
|     30637|
+----------+
only showing top 20 rows



In [24]:
data.select(data['zip_code'].substr(1,5).alias('zip')).show()

+-----+
|  zip|
+-----+
|89518|
|17756|
|90114|
|30645|
|71730|
|74359|
|87713|
|59852|
|82059|
|51655|
|43432|
|40249|
|44793|
|63417|
|18782|
|54903|
|40999|
|42669|
|87064|
|30637|
+-----+
only showing top 20 rows



In [25]:
data = data.withColumn('zip', data['zip_code'].substr(1,5)).drop('zip_code')

In [34]:
data = data.withColumn('zip', data['zip'].cast('int'))

In [35]:
data.head().asDict()

{'Names': 'Cameron Williams',
 'Age': 42.0,
 'Total_Purchase': 11066.8,
 'Account_Manager': 0,
 'Years': 7.22,
 'Num_Sites': 8.0,
 'Onboard_date': datetime.datetime(2013, 8, 30, 7, 0, 40),
 'Location': '10265 Elizabeth Mission Barkerburgh, AK 89518',
 'Company': 'Harvey LLC',
 'Churn': 1,
 'zip': 89518}

In [37]:
data.stat.corr('Years', 'Age')

-0.0345000790268528

In [36]:
data.stat.corr('Churn', 'zip')

-0.07777308812745408

In [61]:
data.select('zip').describe().show()

+-------+------------------+
|summary|               zip|
+-------+------------------+
|  count|               225|
|   mean|52496.671111111114|
| stddev|28652.217580412125|
|    min|               175|
|    max|             99755|
+-------+------------------+

